In [1]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
output_notebook()

import numpy as np

from voltagebudget import util
from voltagebudget import neurons
from fakespikes import util as fsutil

Loading BokehJS ...

# Define input

In [2]:
t = 0.7

k = 20
stim_onset = 0.6
stim_offset = 0.63
stim_rate = 60
dt = 1e-5
seed_stim = 1
ns, ts = util.poisson_impulse(
        t,
        stim_onset,
        stim_offset - stim_onset,
        stim_rate,
        n=k,
        dt=dt,
        seed=seed_stim)

print(">>> {} spikes".format(ts.size))
print(">>> {} population rate".format(ts.size / (stim_offset - stim_onset)))

>>> 35 spikes
>>> 1166.66666667 population rate


In [3]:
p = figure(plot_width=400, plot_height=200)
p.circle(ts*1e3, ns, color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

Shadow voltage response of an LIF neuron to the input:

In [4]:
#set mode
mode = 'regular'
params, w_in, bias, sigma = util.read_modes(mode)

In [5]:
# -
v_in, b_in = neurons.shadow_adex(1, t, ns, ts, w_in=w_in, bias=bias, **params)

times = b_in['times']
m = times > 0.05  # Burn in time

# -
p = figure(plot_width=400, plot_height=200)
p.line(x=times[m], y=v_in[0, :][m], color="black")
# p.x_range = Range1d(0, 0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

ValueError: NeuronGroup size should be at least 1, was 0

# Define computational models

And obseve how they respond to the same input

## Regular

In [ ]:
ns_y, ts_y, budget = neurons.adex(1, t, ns, ts, 
                                  w_in=w_in*3, 
                                  bias=bias, 
                                  sigma=sigma, 
                                  A=.025e-9, f=0, **params)

In [ ]:
p = figure(plot_width=400, plot_height=200)
p.circle(ts_y, ns_y, color="black")
# p.x_range = Range1d(0.0, 0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
times = budget['times']
v = budget['V_m'][0, :]
m = times > 0.05  # Burn in time

p = figure(plot_width=400, plot_height=200)
p.line(x=times[m], y=v[m], color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
times = budget['times']
v = budget['V_comp'][0, :]
m = times > 0.05  # Burn in time

p = figure(plot_width=400, plot_height=200)
p.line(x=times[m], y=v[m], color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
times = budget['times']
v = budget['V_osc'][0, :]
m = times > 0.05  # Burn in time

p = figure(plot_width=400, plot_height=200)
p.line(x=times[m], y=v[m], color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
times = budget['times']
v = budget['V_free'][0, :]
m = times > 0.05  # Burn in time

p = figure(plot_width=400, plot_height=200)
p.line(x=times[m], y=v[m], color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)